In [2]:
from pathlib import Path
from json import loads
subtopicNetworkRootPath = Path("../master-database-files/master-adapted-subtopic-tree-generation/trees/")
assert subtopicNetworkRootPath.exists()

In [3]:
def loadSubtopicTree(name):
    path = subtopicNetworkRootPath / name / "subtopic_tree.json"
    assert path.exists()
    return loads(path.read_text())

In [7]:
tree = loadSubtopicTree("subd")

In [4]:
from json import dumps
def safeVisNetworkJSONToHTMLFile(jsonData, htmlFilePath):
    with open(htmlFilePath, "w") as htmlFile:
        htmlFile.write(
            f"""
            <!DOCTYPE html>
            <html lang="en-US">
            
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>Inline vis</title>
            </head>
            
            <body>
                <div id="mynetwork" style="width:100vw; height:100vh;"></div>
                <script src=" https://cdn.jsdelivr.net/npm/vis-data@7.1.9/peer/umd/vis-data.min.js "></script>
                <script src=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/peer/umd/vis-network.min.js "></script>
                <link href=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/styles/vis-network.min.css " rel="stylesheet">
                <script>
                    var jsonData = {dumps(jsonData)};
                    // create a network
                    var container = document.getElementById("mynetwork");
                    var options = {{}};
                    var network = new vis.Network(container, jsonData, options);
                </script>
            </body>
            
            </html>
            """
        )

In [5]:
def createVisNetworkJSONOfTreeStructure(tree, depth = 3):
    nodes = []
    edges = []
    def addNode(nodeName, children, parentID = None, depth = 0):
        if depth > 0:
            nodeID = len(nodes)
            nodes.append({"id": nodeID, "label": nodeName, "color": {"background": "#E833BD" if parentID is None else "#97C2FC" if children is None else "#97FCB9"}})
            if parentID is not None:
                edges.append({"from": parentID, "to": nodeID, "color": "#97C2FC" if children is None else "#97FCB9", "value" : 1 if children is None else 3 + depth/2})
            if children:
                for childName, child in children.items():
                    addNode(childName, child, nodeID, depth - 1)
    for nodeName, node in tree.items():
        addNode(nodeName, node, depth = depth)
    return {"nodes": nodes, "edges": edges}

In [10]:
vNJ = createVisNetworkJSONOfTreeStructure(tree, 8)
safeVisNetworkJSONToHTMLFile(vNJ, "output.html")

In [8]:
subd10Tree = loads(Path("../master-database-files/master-experimental/subtopic_tree_comparison/trees/subd10gpt4/subtopic_tree.json").read_text())
safeVisNetworkJSONToHTMLFile(createVisNetworkJSONOfTreeStructure(subd10Tree, 30), "subd10.html")

In [9]:
subd20Tree = loads(Path("../master-database-files/master-experimental/subtopic_tree_comparison/trees/subd20gpt4/subtopic_tree.json").read_text())
safeVisNetworkJSONToHTMLFile(createVisNetworkJSONOfTreeStructure(subd20Tree, 30), "subd20.html")